In [53]:
import collections
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [30]:
# Load Preprocessed Silo Data 
silo_paths = [
    '../datasets/diabetes/processed_silos/hospital_1.csv',
    '../datasets/diabetes/processed_silos/hospital_2.csv',
    '../datasets/diabetes/processed_silos/hospital_3.csv',
    '../datasets/diabetes/processed_silos/hospital_4.csv',
    '../datasets/diabetes/processed_silos/hospital_5.csv'
]

def load_silo_data(path):
    df = pd.read_csv(path)
    
    # Correct binary label: 0 = NO, 1 = <30 or >30
    df['readmitted_binary'] = df['readmitted'].apply(lambda x: 0 if x == 0 else 1)

    # Drop target and leaky column(s)
    df = df.drop(columns=['discharge_disposition_id'])

    y = df['readmitted'].values
    X = df.drop(columns=['readmitted']).values
    return tf.data.Dataset.from_tensor_slices((X.astype(np.float32), y.astype(np.int32))).batch(32)

datasets = [load_silo_data(p) for p in silo_paths]

In [47]:
# --- Define Model Function ---
def create_keras_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(datasets[0].element_spec[0].shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=datasets[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

In [48]:
# --- Federated Averaging Process ---
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=0.01),
    server_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=1.0)
)

state = iterative_process.initialize()

2025-06-10 20:04:22.623431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-06-10 20:04:22.623504: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-06-10 20:04:22.623743: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-06-10 20:04:22.624464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-06-10 20:04:22.624511: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

In [55]:
# --- Federated Training Loop ---
NUM_ROUNDS = 10
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, datasets)
    print(f'Round {round_num}, Metrics={metrics}')


2025-06-10 20:12:03.234480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-06-10 20:12:03.234552: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-06-10 20:12:03.234740: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-06-10 20:12:03.235263: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-06-10 20:12:03.235282: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

Round 1, Metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.3524), ('loss', -3.3109121), ('num_examples', 30000), ('num_batches', 940)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 2, Metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.35326666), ('loss', -7.44593), ('num_examples', 30000), ('num_batches', 940)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 3, Metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.35326666), ('loss', -7.5886307), ('num_examples', 30000), ('num_batches', 940)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_fin

In [57]:
import datetime
import os
import json
from tensorflow_federated.python.learning.optimizers import sgdm

# --- Evaluation Function ---
def evaluate_model(state, model_fn, test_data):
    """Evaluate the current model on test data."""
    model = model_fn()
    model.from_weights(state.global_model_weights)
    evaluation_metrics = {}
    
    for client_id, client_data in enumerate(test_data):
        metrics = model.evaluate(client_data, return_dict=True)
        for metric_name, metric_value in metrics.items():
            if metric_name not in evaluation_metrics:
                evaluation_metrics[metric_name] = []
            evaluation_metrics[metric_name].append(float(metric_value))
    
    # Calculate average metrics across all clients
    avg_metrics = {f'avg_{k}': np.mean(v) for k, v in evaluation_metrics.items()}
    return {**evaluation_metrics, **avg_metrics}


In [58]:
# --- Logging Setup ---
def setup_logging():
    """Create logging directory and config."""
    log_dir = f"logs/fed_avg_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(log_dir, exist_ok=True)
    
    config = {
        "model": "Sequential_Dense64",
        "client_optimizer": "SGDM_lr0.01",
        "server_optimizer": "SGDM_lr1.0",
        "num_rounds": NUM_ROUNDS,
        "batch_size": 32,
        "creation_time": datetime.datetime.now().isoformat()
    }
    
    with open(f"{log_dir}/config.json", 'w') as f:
        json.dump(config, f, indent=2)
    
    return log_dir

In [61]:
# --- Modified Training Loop with Logging ---
log_dir = setup_logging()
training_history = []

for round_num in range(1, NUM_ROUNDS + 1):
    # Train for one round
    state, metrics = iterative_process.next(state, datasets)
    
    # Evaluate on all client datasets
    eval_metrics = evaluate_model(state, model_fn, datasets)
    
    # Combine training and evaluation metrics
    round_metrics = {
        "round": round_num,
        "train": {k: float(v) for k, v in metrics['client_work']['train'].items()},
        "eval": eval_metrics,
        "timestamp": datetime.datetime.now().isoformat()
    }
    
    # Save metrics to history and log file
    training_history.append(round_metrics)
    with open(f"{log_dir}/metrics.json", 'w') as f:
        json.dump(training_history, f, indent=2)
    
    # Save model periodically
    if round_num % 5 == 0 or round_num == NUM_ROUNDS:
        save_model(state, round_num, log_dir)
    
    print(f"Round {round_num} completed")
    print(f"Training metrics: {round_metrics['train']}")
    print(f"Evaluation metrics: {round_metrics['eval']['avg_binary_accuracy']:.4f}")

AttributeError: '_KerasModel' object has no attribute 'from_weights'

In [ ]:
# --- Final Model Export ---
def export_keras_model(state, model_fn, log_dir):
    """Export the final model as a Keras SavedModel."""
    keras_model = create_keras_model()
    keras_model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    # Convert TFF weights to Keras model weights
    tff_weights = state.global_model_weights
    keras_model.set_weights([w.numpy() for w in tff_weights.trainable])
    
    # Save as SavedModel
    export_path = f"{log_dir}/final_model"
    keras_model.save(export_path)
    print(f"Model exported to {export_path}")

# Export the final model
export_keras_model(state, model_fn, log_dir)

print("Training complete! All artifacts saved to:", log_dir)